In [ ]:


import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import (Input, Conv2D, Conv2DTranspose,
                                     Concatenate, BatchNormalization,
                                     Activation, Lambda)
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (ModelCheckpoint, EarlyStopping,
                                        LearningRateScheduler)
from sklearn.model_selection import train_test_split

# ------------------------------------------------------------------
# 1. Data‑loading stub  ⇢  REPLACE with real loader
# ------------------------------------------------------------------



# ------------------------------------------------------------------
# 2. DenseNet121‑U‑Net model
# ------------------------------------------------------------------
def densenet121_unet(input_size=(256, 256, 3),
                     freeze_encoder=True,
                     decoder_filters=(512, 256, 128, 64),
                     upsample_mode="transpose",
                     final_activation="sigmoid"):
    """
    Build a DenseNet121‑based U‑Net.
    """
    # ---- Encoder --------------------------------------------------
    dense = DenseNet121(weights="imagenet",
                        include_top=False,
                        input_tensor=Input(shape=input_size))

    # DenseNet feature maps for skip connections
    skip_names = [
        "conv1/relu",      #  64×64×64    (DenseNet downsamples early)
        "pool2_relu",      #  32×32×128
        "pool3_relu",      #  16×16×256
        "pool4_relu"       #   8×8×512
    ]
    encoder_output = dense.output            # 8×8×1024 (after final dense block)

    if freeze_encoder:
        for layer in dense.layers:
            layer.trainable = False

    # ---- Decoder helper ------------------------------------------
    def upsample(x, filters, name):
        if upsample_mode == "transpose":
            return Conv2DTranspose(filters, 2, strides=2, padding="same",
                                   name=f"{name}_up")(x)
        # bilinear upsample + conv
        x = Lambda(lambda t: tf.image.resize(t,
                                             (t.shape[1] * 2, t.shape[2] * 2),
                                             method="bilinear"),
                   name=f"{name}_resize")(x)
        return Conv2D(filters, 3, padding="same", name=f"{name}_conv")(x)

    # ---- Decoder -------------------------------------------------
    x = encoder_output
    for i, (skip_name, f) in enumerate(zip(reversed(skip_names), decoder_filters), 1):
        x = upsample(x, f, name=f"dec{i}")
        skip = dense.get_layer(skip_name).output
        x = Concatenate(name=f"concat{i}")([x, skip])

        # Two conv‑BN‑ReLU layers
        for j in range(2):
            x = Conv2D(f, 3, padding="same", name=f"dec{i}_conv{j+1}")(x)
            x = BatchNormalization(name=f"dec{i}_bn{j+1}")(x)
            x = Activation("relu", name=f"dec{i}_act{j+1}")(x)

    # Final upsample to 256×256
    x = upsample(x, decoder_filters[-1] // 2, name="dec_final")

    mask = Conv2D(1, 1, activation=final_activation,
                  padding="same", name="mask")(x)

    mask = Lambda(lambda t: tf.image.resize(t,
                                            (input_size[0], input_size[1]),
                                            method="bilinear"),
                  name="identity_resize")(mask)

    model = Model(inputs=dense.input, outputs=mask, name="DenseNet121_UNet")
    model.compile(optimizer="adam",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model


# ------------------------------------------------------------------
# 3. Paths and hyper‑parameters
# ------------------------------------------------------------------
IMAGE_DIR = "datasets2/images"
MASK_DIR  = "datasets2/masks"
TILE_SIZE = 256
BATCH_SIZE = 32
EPOCHS = 100
size = 256
# ------------------------------------------------------------------
# 4. Prepare data
# ------------------------------------------------------------------
X, y = load_data(IMAGE_DIR, MASK_DIR, TILE_SIZE)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42)

# Add channel dimension to masks
y_train = y_train[..., np.newaxis]
y_val   = y_val[..., np.newaxis]
y_test  = y_test[..., np.newaxis]

# Data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8, 1.2]
)

# ------------------------------------------------------------------
# 5. Callbacks
# ------------------------------------------------------------------
def lr_schedule(epoch):
    base_lr = 1e-4
    decay   = 0.9
    return base_lr * (decay ** (epoch // 10))

lr_scheduler = LearningRateScheduler(lr_schedule)

checkpointer  = ModelCheckpoint("best_densenet_unet.h5",
                                monitor="val_loss",
                                save_best_only=True,
                                mode="min")

early_stop   = EarlyStopping(monitor="val_loss",
                             patience=5,
                             mode="min",
                             verbose=1)

callbacks = [lr_scheduler, checkpointer, early_stop]

# ------------------------------------------------------------------
# 6. Build and train model
# ------------------------------------------------------------------
model = densenet121_unet(input_size=(TILE_SIZE, TILE_SIZE, 3),
                         freeze_encoder=True)

history = model.fit(datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                    epochs=EPOCHS,
                    validation_data=(X_val, y_val),
                    callbacks=callbacks)

# ------------------------------------------------------------------
# 7. Evaluate and save
# ------------------------------------------------------------------
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f} | Test Accuracy: {acc:.4f}")

model.save("forest_detection_densenet_unet_full.h5")

# ------------------------------------------------------------------
# 8. Plot losses
# ------------------------------------------------------------------
plt.figure(figsize=(6, 4))
plt.plot(history.history["loss"],     label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.tight_layout()
plt.savefig("densenet_unet_training_curve.png")
plt.close()
print("Training curve saved to densenet_unet_training_curve.png")

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from tensorflow.keras.models import load_model

from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Load the saved model
loaded_model = load_model('best_densenet_unet.h5')

# Predictions on the test set
y_pred = loaded_model.predict(X_test)

# Threshold the predictions to get binary values (0 or 1)
y_pred_binary = (y_pred > 0.5).astype(int)

# Flatten the arrays for metrics calculation
y_test_flat = y_test.flatten()
y_pred_flat = y_pred_binary.flatten()

# Calculate and print accuracy
accuracy = accuracy_score(y_test_flat, y_pred_flat)
print(f'Accuracy: {accuracy:.4f}')

# Calculate and print F1 score
f1 = f1_score(y_test_flat, y_pred_flat)
print(f'F1 Score: {f1:.4f}')

# Calculate and print precision
precision = precision_score(y_test_flat, y_pred_flat)
print(f'Precision: {precision:.4f}')

# Calculate and print recall
recall = recall_score(y_test_flat, y_pred_flat)
print(f'Recall: {recall:.4f}')

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test_flat, y_pred_flat)
print('Confusion Matrix:')
print(conf_matrix)

# Calculate mean intersection over union (mIoU)
intersection = np.sum(np.logical_and(y_test_flat, y_pred_flat))
union = np.sum(np.logical_or(y_test_flat, y_pred_flat))
miou = intersection / union
print(f'Mean Intersection over Union (mIoU): {miou:.4f}')


# --- AUC‑ROC --------------------------------------------------------------
# 1‑D‑ify the ground‑truth mask and the model’s probability output
y_test_prob = y_test.flatten()
y_pred_prob = y_pred.flatten()       # <‑‑ raw probabilities, NOT thresholded

# Compute AUC
auc = roc_auc_score(y_test_prob, y_pred_prob)
print(f'AUC‑ROC: {auc:.4f}')

# Optional: plot the ROC curve
fpr, tpr, _ = roc_curve(y_test_prob, y_pred_prob)

plt.figure()
plt.plot(fpr, tpr, linewidth=2, label=f'ROC curve (AUC = {auc:.3f})')
plt.plot([0, 1], [0, 1], linestyle='--', linewidth=1)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.show()